# Building LLM applications: Notebook 01

# Langchain basics

## Initialize

In [1]:
import os
import dotenv

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'

In [3]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: http://kqrw311-g5-12xlarge-a.img.astrazeneca.net:8080


## Excercise 01: Connect to an LLM

Connect to an LLM and make a simple request (e..g translate a senetence)

```
Translate to French: I like coffee.
```

Take a look at the LLM parameters, and full response 

In [4]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = "Translate to French: I like coffee."

# Invoke the LLM
ret = llm.invoke(messages)
ret.pretty_print()

================================== Ai Message ==================================

Here is the translation:

J'aime le café.

Or, in a more informal tone:

Je bois beaucoup de café.

Or, in a more casual tone:

C'est mon café préféré!

(Note: The first option is formal, the second is informal and polite, and the third is very casual and friendly.)


In [5]:
# Show details of the response
print(ret.model_dump_json(indent=2))

{
  "content": "Here is the translation:\n\nJ'aime le café.\n\nOr, in a more informal tone:\n\nJe bois beaucoup de café.\n\nOr, in a more casual tone:\n\nC'est mon café préféré!\n\n(Note: The first option is formal, the second is informal and polite, and the third is very casual and friendly.)",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "llama3.2:3b-instruct-fp16",
    "created_at": "2025-01-16T12:33:15.768067591Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 7661757204,
    "load_duration": 20412346,
    "prompt_eval_count": 33,
    "prompt_eval_duration": 166000000,
    "eval_count": 66,
    "eval_duration": 7473000000,
    "message": {
      "role": "assistant",
      "content": "",
      "images": null,
      "tool_calls": null
    }
  },
  "type": "ai",
  "name": null,
  "id": "run-ab27dc7e-a409-4791-a9ef-321d46cedb02-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 33,
 

## Excercise 02: Prompt template

In [6]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable'
ret = runnable.invoke({"sentence": "I like coffee."})
ret.pretty_print()

================================== Ai Message ==================================

Je aime le café.


## Excercise 03: Streaming

In [7]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable' as a stream, print each block as it arrives
blocks = []
for b in runnable.stream({"sentence": "I like coffee."}):
    print(b.content, end='')
    blocks.append(b)

print(f"\n----\nReceived {len(blocks)} blocks")

Je aime le café.
----
Received 6 blocks


In [8]:
# What does a block look like
print(blocks[0])

content='Je' additional_kwargs={} response_metadata={} id='run-7b7f84d9-db06-4e13-bb13-5395ea75b43b'


## Excercise 04: Roles

In [9]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = [
    ('system', "You are helping to translate the user's inputs from english to french"),
    ('human', "I like coffee")
]

ret = llm.invoke(messages)
ret.pretty_print()

================================== Ai Message ==================================

J'aime le café. 

Note: I used a more formal way of saying "I like coffee" in French, which is suitable for writing or speaking with someone you don't know well.

If you want to say it in an informal way (with friends or family), you can use:

Je aime un café, ça fait plaisir!

This means "I love a cup of coffee, it's nice!"


## Excercise 04: Message prompt templates with roles

In [10]:
# Create the LLM object
messages_template = [
    ('system', "You are helping to translate the user's inputs from English to {language}"),
    ('human', "{sentence}")
]

llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)
prompt = ChatPromptTemplate.from_messages(messages=messages_template)
runnable = prompt | llm

# Run the 'runnable'
ret = runnable.invoke({"language": "French", "sentence": "I like coffee."})
ret.pretty_print()

================================== Ai Message ==================================

Je aime le café.
